In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from joblib import Parallel, delayed
import matplotlib.pyplot as plt

In [ ]:
image_size = (64, 64)
batch_size = 32
epochs = 15
num_classes = 4
data_dir = "/home/awanardy/Documents/Projek/KlasifikasiHiragana/hiragana_katakana"

In [ ]:
def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, image_size)
    img = img / 255.0
    return img

In [ ]:
def load_data(data_dir):
    images = []
    labels = []
    for label, folder_name in enumerate(["hiragana_hitam", "hiragana_putih", "katakana_hitam", "katakana_putih"]):
        folder_path = os.path.join(data_dir, folder_name)
        image_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".png")]
        images.extend(Parallel(n_jobs=-1)(delayed(load_and_preprocess_image)(image_path) for image_path in image_paths))
        labels.extend([label] * len(image_paths))
    return np.array(images), np.array(labels)

In [ ]:
X, y = load_data(data_dir)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [ ]:
with open('model_hiragana_katakana.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Akurasi pada data uji:", accuracy)

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Akurasi Model')
plt.xlabel('Epoch')
plt.ylabel('Akurasi')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Model')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()